## Analyzing historical mining data with Amazon Bedrock

This notebook demonstrates how a large language model like Anthropic Claude can be used to analyze unstructured text found in a mine owners report from 1939 [1].

[1] [CyrusPima528.pdf](https://minedata.azgs.arizona.edu/report/cyrus). 2011-01-1062, ADMMR mining collection, Arizona Geological Survey

#### Prerequisites

This notebook is designed to be run in a [Amazon SageMaker Notebook Instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html). Standalone use may require installation of additional Python packages or code modifications. If you're starting a new notebook, we recommend that you create the notebook in [Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html) instead of launching a notebook instance from the Amazon SageMaker console.

Access to Amazon Bedrock foundation models isn't granted by default. See [Manage access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) for information on how to enable access to Anthropic Claude 3 Sonnet before running these examples.

#### Permissions

To use the Converse API, you call the Converse or ConverseStream operations to send messages to a model. To call Converse, you require permission for the bedrock:InvokeModel operation. To call ConverseStream, you require permission for the bedrock:InvokeModelWithResponseStream operation.

For more information see [Identity and access management for Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html)

#### Clean-up

Notebook instances are compute instances running the Jupyter notebook app. You are charged for the instance type you choose, based on the duration of use. Stop Notebook instances when not in use to save cost and delete instances when no longer required.

### The Converse API
The [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) provides consistent API that works with all Amazon Bedrock models that support messages. This means you can write code once and use it with different models. Should a model have unique inference parameters, the Converse API also allows you to pass those unique parameters in a model specific structure.

To use the Converse API, you use the Converse or ConverseStream (for streaming responses) operations to send messages to a model. A conversation is a series of messages between the user and the model and you maintain conversation over many turns by storing each message of the conversation in an array of [Message](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Message.html) objects. You start a conversation by sending a message as a user (user role) to the model. The model, acting as an assistant (assistant role), then generates a response that it returns in a message. 

In this example we will using Anthropic Claude 3 Sonnet via the Converse API to perform single-shot inference.

**Note**: Amazon Bedrock doesn't store any text, images, or documents that you provide as content. The data is only used to generate the response. When using Converse API, you must use an uncompressed and decoded document that is less than 4.5 MB in size.

### Constructing a Message and invoking the Converse API
boto3 provides [Converse](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html) to interact with an Amazon Bedrock model using the Converse API. At a minimum the request must include the **modelId** to invoke and a list of **messages**:

```python
response = bedrock_client.converse(
    modelId = 'string',
    messages = [
        {
        'role': 'user'|'assistant',
        'content': [
            {
                'text': 'string',
                'document': {
                    'format': 'pdf'|'csv'|'doc'|'docx'|'xls'|'xlsx'|'html'|'txt'|'md',
                    'name': 'string',
                    'source': {
                        'bytes': b'bytes'
                    }
                }
            }
            ]
        }
    ]
)
```
Each input message includes a role and content, such as a conversation turn or document. To submit a document for analysis we provide a [DocumentBlock](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_DocumentBlock.html) object. The file contents are included in a [DocumentSource](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_DocumentSource.html) object as Base64-encoded binary data. When using an AWS SDK it will handle encoding of the bytes to Base64 so we open and read our file in binary mode.

The Converse API returns data in JSON format. See the [Response Elements](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html#API_runtime_Converse_ResponseElements) documentation for more information the structure of returned data.

### Create a bedrock-runtime client

In [ ]:
import logging
import boto3
import base64
import json

# Create the Bedrock client
bedrock_client = boto3.client(service_name="bedrock-runtime")

# See https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html for information on model compatibility
MODEL_NAME = "anthropic.claude-3-sonnet-20240229-v1:0"

# Modify path if file is not uploaded to the base folder of the Notebook instance
INPUT_DOCUMENT = open('/home/ec2-user/SageMaker/CyrusPima528.pdf', 'r+b').read()

### Generating a summary
Let's ask the model to create a summary of the document. 

An LLM generates text by making predictions of the likelihood of one word appearing after another based on a a small number of input words, in this case our historical document. The likelihood is based on a mathematical relationship established between words the model is exposed to during training. 

This means that the quality of the output generated is dependent on the data the model was exposed to during training. Domain specific applications like mining can challenge model capabilities and techniques such as [retrieval augmented generation](https://aws.amazon.com/what-is/retrieval-augmented-generation/) and [fine tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-fine-tuning.html) exist to improve the quality and traceability of model output.

The output the model is generating is completely new - there's no direct relationship between the summary and the source, and the results may differ each time the model is run. Inaccuracies, also known as hallucinations, may occur and the output generated by an LLM should never be used without appropriate validation of generated data.

In [ ]:
messages = [{
    "role": "user",
    "content": [
        {
            "text": "Provide a summary of the document"
        },
        {
        "document": {
            "name": "CyrusPima528",
            "format": "pdf",
            "source": {
                "bytes": INPUT_DOCUMENT
            }
        }
        }
    ]
}]

response = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages
)

print(response['output']['message']['content'][0]['text'])

### Prompt engineering
Prompts are the set of inputs provided by you that guide LLMs on Amazon Bedrock to generate an appropriate response or output for a given task or instruction. The more specific and detailed our request the better the generated response will be.

The prompt provided to the model in the previous example was broad and leaves a lot of opportunity for the output to vary. Let's generate a concise document summary by providing a more detailed prompt:

In [ ]:
messages = [{
    "role": "user",
    "content": [
        {
            "text": """Provide a summary of the document. 
                       Include the primary mineral being mined and whether mining has occured."""
        },
        {
        "document": {
            "name": "CyrusPima528",
            "format": "pdf",
            "source": {
                "bytes": INPUT_DOCUMENT
            }
        }
        }
    ]
}]

response = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages
)

print(response['output']['message']['content'][0]['text'])

### System prompts
A system prompt lets you provide context and instructions to the model, such as specifying a particular goal or role. This technique is known as role prompting and can provide enhanced accuracy, a role-specific communication style, and improved focus for domain specific applications.

Let's provide a system prompt that gives the model a more mining specific persona that provides a more technical output:

In [ ]:
messages = [{
    "role": "user",
    "content": [
        {
            "text": """Provide a concise summary of the document.
                       Include the primary mineral being mined and detail on mineralization encountered."""
        },
        {
        "document": {
            "name": "CyrusPima528",
            "format": "pdf",
            "source": {
                "bytes": INPUT_DOCUMENT
            }
        }
        }
    ]
}]

system = [{
        "text": """You are a geologist for a mining company"""
    }]

response1 = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages
)

response2 = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages,
    system = system
)

print("Without system prompt: \n" + response1['output']['message']['content'][0]['text'])
print("\nWith system prompt: \n" + response2['output']['message']['content'][0]['text'])

The results have improved but the model is still quite chatty, a known trait of Claude. Let's be more specific:

In [ ]:
messages = [{
    "role": "user",
    "content": [
        {
            "text": """Provide a compact and concise analysis of the document in the following format:

                       Name. Location. Mineralization. Total mineral recovered."""
        },
        {
        "document": {
            "name": "CyrusPima528",
            "format": "pdf",
            "source": {
                "bytes": INPUT_DOCUMENT
            }
        }
        }
    ]
}]

system = [{
        "text": """You are a geologist for a mining company"""
    }]

response1 = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages
)

response2 = bedrock_client.converse(
    modelId = MODEL_NAME,
    messages = messages,
    system = system
)

print("Without system prompt: \n" + response1['output']['message']['content'][0]['text'])
print("\nWith system prompt: \n" + response2['output']['message']['content'][0]['text'])

### Outputting a summary in JSON format

Now that we have the model getting to the point, we could use this summarization capability as part of an automated document processing pipeline. JSON is a widely supported and open standard file format for data interchange and allows for easy integration with other systems.

While Claude doesn't have a formal JSON mode, we can still ask model to format its output in JSON:

In [ ]:
message = {
    "role": "user",
    "content": [
        {
            "text": """Provide a summary of the document and provide the output in JSON
            
                       Follow the format provided below:

                        file_name - filename
                        mine_name - mine name
                        summary - a single line analysis in the following format: Name, Location, Mineralization, Total mineral recovered.
                        location - location
                        first_mined - date of first mining activity. n/a if unknown
                        recovery - total ore recovered. 0 if unknown
                        primary_mineralization - primary mineral
                        primary_concentration - primary mineral concentration
                        secondary_mineralization - an array of secondary minerals identified
                    """
        },
        {
        "document": {
            "name": "CyrusPima528",
            "format": "pdf",
            "source": {
                "bytes": INPUT_DOCUMENT
            }
        }
        }
    ]
}

system = [ {
    "text": """You are a geologist for a mining company"""
    } ]

messages = [message]

response = bedrock_client.converse (
    modelId = MODEL_NAME,
    messages = messages,
    system = system
)

print(response['output']['message']['content'][0]['text'])

While this approach works, it's not guaranteed to consistently generate valid JSON. Claude's tool use functionality could be used to generate output based on a format we define. See [Forcing JSON with tool use](https://github.com/aws-samples/prompt-engineering-with-anthropic-claude-v-3/blob/main/10_2_2_Tool_Use_for_Structured_Outputs.ipynb) for an example of this technique.

### Summary
This notebook demonstrates how a multi-modal LLM can be used to summarize historic exploration and mining records, reducing the time required to sort and prepare data for further analysis. Care must be taken to verify output accuracy and account for hallucinations, areas where techniques such as retrieval augmented generation can help.